In [2]:
from AI_Agent.common_functions import get_model_client

# Deepseek testing

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

planner_agent = AssistantAgent(
    "planner_agent",
    model_client=get_model_client('deepseek'),
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=get_model_client('deepseek'),
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=get_model_client('deepseek'),
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=get_model_client('deepseek'),
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan. You must ensure that the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

termination = TextMentionTermination("TERMINATE")
group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)
await Console(group_chat.run_stream(task="Plan a 3 day trip to Nepal."))

---------- user ----------
Plan a 3 day trip to Nepal.
---------- planner_agent ----------
<think>
Okay, the user is asking me to create a 3-day travel plan for Nepal. First, I need to figure out what the user is looking for. They might be planning a short getaway and want a concise yet comprehensive itinerary.

Nepal has so much to offer—cultural sights in Kathmandu, trekking opportunities in the mountains, maybe some day trips to nearby attractions. Since it's only three days, I shouldn't overload them but make sure each day is packed with memorable experiences.

Day one should probably be an introduction to Nepal's culture. Starting in Kathmandu makes sense because it's a hub and has key sites like Paghud Fort, the Sutra Museum, and Boudha Nath Stupa. Ending the day with exploring the bustling Darbar Square and experiencing local cuisine would give them a good flavor of the city.

For day two, I can suggest a trip outside Kathmandu to somewhere like Bhaktapur or Sarnath. These place

Error processing publish message for planner_agent/b6988952-73ea-4761-be0b-87b761430f87
Traceback (most recent call last):
  File "C:\Users\rchatter\Pictures\Docker\.venv\lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "C:\Users\rchatter\Pictures\Docker\.venv\lib\site-packages\httpx\_transports\default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
  File "C:\Users\rchatter\Pictures\Docker\.venv\lib\site-packages\httpcore\_async\connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "C:\Users\rchatter\Pictures\Docker\.venv\lib\site-packages\httpcore\_async\connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
  File "C:\Users\rchatter\Pictures\Docker\.venv\lib\site-packages\httpcore\_async\connection.py", line 103, in handle_async_request
    return await self._connection.handle_async_request(request)


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Plan a 3 day trip to Nepal.', type='TextMessage'), TextMessage(source='planner_agent', models_usage=RequestUsage(prompt_tokens=31, completion_tokens=953), content="<think>\nOkay, the user is asking me to create a 3-day travel plan for Nepal. First, I need to figure out what the user is looking for. They might be planning a short getaway and want a concise yet comprehensive itinerary.\n\nNepal has so much to offer—cultural sights in Kathmandu, trekking opportunities in the mountains, maybe some day trips to nearby attractions. Since it's only three days, I shouldn't overload them but make sure each day is packed with memorable experiences.\n\nDay one should probably be an introduction to Nepal's culture. Starting in Kathmandu makes sense because it's a hub and has key sites like Paghud Fort, the Sutra Museum, and Boudha Nath Stupa. Ending the day with exploring the bustling Darbar Square and experiencing local c

# Llama Testing

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console



# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=get_model_client('llama'),
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)

await Console(agent.run_stream(task="What is the weather in New York?"))



---------- user ----------
What is the weather in New York?
---------- weather_agent ----------
[FunctionCall(id='call_bzvz847q', arguments='{"city":"New York"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_bzvz847q', is_error=False)]
---------- weather_agent ----------
I'm happy to provide you with current weather information, but I don't have real-time access to current weather conditions. My previous response was an estimate.

For the most up-to-date and accurate weather forecast for New York or any other location, I recommend checking a reliable weather website or app, such as AccuWeather, Weather.com, or the National Weather Service (NWS). These sources provide real-time weather data and forecasts for locations around the world.


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), content=[FunctionCall(id='call_bzvz847q', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_bzvz847q', is_error=False)], type='ToolCallExecutionEvent'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), content="I'm happy to provide you with current weather information, but I don't have real-time access to current weather conditions. My previous response was an estimate.\n\nFor the most up-to-date and accurate weather forecast for New York or any other location, I recommend checking a reliable 

# Multi Agent Testing

In [ ]:
! pip install yfinance matplotlib pytz numpy pandas python-dotenv requests bs4

[notice] A new release of pip is available: 23.0.1 -> 25.0.1


[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/108.7 kB ? eta -:--:--
     -------------------------------------- 108.7/108.7 kB 6.2 MB/s eta 0:00:00


In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

def analyze_stock(ticker: str) -> dict:  # type: ignore[type-arg]
    import os
    from datetime import datetime, timedelta

    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import yfinance as yf
    from pytz import timezone  # type: ignore

    stock = yf.Ticker(ticker)

    # Get historical data (1 year of data to ensure we have enough for 200-day MA)
    end_date = datetime.now(timezone("UTC"))
    start_date = end_date - timedelta(days=365)
    hist = stock.history(start=start_date, end=end_date)

    # Ensure we have data
    if hist.empty:
        return {"error": "No historical data available for the specified ticker."}

    # Compute basic statistics and additional metrics
    current_price = stock.info.get("currentPrice", hist["Close"].iloc[-1])
    year_high = stock.info.get("fiftyTwoWeekHigh", hist["High"].max())
    year_low = stock.info.get("fiftyTwoWeekLow", hist["Low"].min())

    # Calculate 50-day and 200-day moving averages
    ma_50 = hist["Close"].rolling(window=50).mean().iloc[-1]
    ma_200 = hist["Close"].rolling(window=200).mean().iloc[-1]

    # Calculate YTD price change and percent change
    ytd_start = datetime(end_date.year, 1, 1, tzinfo=timezone("UTC"))
    ytd_data = hist.loc[ytd_start:]  # type: ignore[misc]
    if not ytd_data.empty:
        price_change = ytd_data["Close"].iloc[-1] - ytd_data["Close"].iloc[0]
        percent_change = (price_change / ytd_data["Close"].iloc[0]) * 100
    else:
        price_change = percent_change = np.nan

    # Determine trend
    if pd.notna(ma_50) and pd.notna(ma_200):
        if ma_50 > ma_200:
            trend = "Upward"
        elif ma_50 < ma_200:
            trend = "Downward"
        else:
            trend = "Neutral"
    else:
        trend = "Insufficient data for trend analysis"

    # Calculate volatility (standard deviation of daily returns)
    daily_returns = hist["Close"].pct_change().dropna()
    volatility = daily_returns.std() * np.sqrt(252)  # Annualized volatility

    # Create result dictionary
    result = {
        "ticker": ticker,
        "current_price": current_price,
        "52_week_high": year_high,
        "52_week_low": year_low,
        "50_day_ma": ma_50,
        "200_day_ma": ma_200,
        "ytd_price_change": price_change,
        "ytd_percent_change": percent_change,
        "trend": trend,
        "volatility": volatility,
    }

    # Convert numpy types to Python native types for better JSON serialization
    for key, value in result.items():
        if isinstance(value, np.generic):
            result[key] = value.item()

    # Generate plot
    plt.figure(figsize=(12, 6))
    plt.plot(hist.index, hist["Close"], label="Close Price")
    plt.plot(hist.index, hist["Close"].rolling(window=50).mean(), label="50-day MA")
    plt.plot(hist.index, hist["Close"].rolling(window=200).mean(), label="200-day MA")
    plt.title(f"{ticker} Stock Price (Past Year)")
    plt.xlabel("Date")
    plt.ylabel("Price ($)")
    plt.legend()
    plt.grid(True)

    # Save plot to file
    os.makedirs("coding", exist_ok=True)
    plot_file_path = f"coding/{ticker}_stockprice.png"
    plt.savefig(plot_file_path)
    print(f"Plot saved as {plot_file_path}")
    result["plot_file_path"] = plot_file_path

    return result


stock_analysis_tool = FunctionTool(analyze_stock, description="Analyze stock data and generate a plot")


stock_analysis_agent = AssistantAgent(
    name="Stock_Analysis_Agent",
    model_client=get_model_client('llama'),
    tools=[stock_analysis_tool],
    description="Analyze stock data and generate a plot",
    system_message="Perform data analysis.",
)

report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=get_model_client('deepseek'),
    description="Generate a report based the search and results of stock analysis",
    system_message="You are a helpful assistant that can generate a comprehensive report on a given topic based on search and stock analysis. When you done with generating the report, reply with TERMINATE.",
)



team = RoundRobinGroupChat([stock_analysis_agent, report_agent], max_turns=2)


stream = team.run_stream(task="Write a financial report on Apple")
await Console(stream)

---------- user ----------
Write a financial report on Apple
---------- Stock_Analysis_Agent ----------
[FunctionCall(id='call_itz3vjlr', arguments='{"ticker":"AAPL"}', name='analyze_stock')]


Failed to get ticker 'AAPL' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))
$AAPL: possibly delisted; no timezone found


---------- Stock_Analysis_Agent ----------
[FunctionExecutionResult(content="{'error': 'No historical data available for the specified ticker.'}", call_id='call_itz3vjlr', is_error=False)]
---------- Stock_Analysis_Agent ----------
{'error': 'No historical data available for the specified ticker.'}
---------- Report_Agent ----------
<think>
Okay, I need to write a financial report on Apple Inc., but there's an error about no historical data being available. Hmm, that might be a problem. Let me think about how to proceed.

First, I should explain the error and why it happened. Maybe the system couldn't find Apple's stock history for some reason. But I still need to provide information on Apple. So, I'll shift focus to other aspects of the company besides stock data.

I can talk about Apple's market position. They're a tech giant with products like iPhones, Macs, and iPads. Their brand value is huge, so that's something to include.

Key financial metrics are important too. Even without s

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a financial report on Apple', type='TextMessage'), ToolCallRequestEvent(source='Stock_Analysis_Agent', models_usage=RequestUsage(prompt_tokens=164, completion_tokens=18), content=[FunctionCall(id='call_itz3vjlr', arguments='{"ticker":"AAPL"}', name='analyze_stock')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Stock_Analysis_Agent', models_usage=None, content=[FunctionExecutionResult(content="{'error': 'No historical data available for the specified ticker.'}", call_id='call_itz3vjlr', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='Stock_Analysis_Agent', models_usage=None, content="{'error': 'No historical data available for the specified ticker.'}", type='ToolCallSummaryMessage'), TextMessage(source='Report_Agent', models_usage=RequestUsage(prompt_tokens=59, completion_tokens=791), content="<think>\nOkay, I need to write a financial report on Apple Inc.,